# Analysis Notebook for the scraper project

In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import os
import sys
import re

In [2]:
# prevent hiding of columns
pd.options.display.max_columns = None

# prevent cropping of lng strings for display
pd.options.display.max_colwidth = 300
pd.set_option('display.max_rows', 500)

In [3]:
# no wrapping of long strings

In [4]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [90]:
root = Path(os.getcwd()).parent
fname = root / 'data'  /  'room_database.csv'
fname = root / 'data'  /  'processed_data.csv'
fname = root / 'data'  /  'post_processed_data.csv'
# fname = root / 'data'  / 'searchresults' / 'results_2020_10_24__13_49_00.csv'
df=pd.read_csv(fname)
print(df.columns)
df.head(5)#.sort_values(by='timestamp',ascending=False)

Index(['timestamp', 'scrape_ts', 'domain', 'rooms', 'cost', 'address',
       'duration', 'bid_ask', 'href', 'title', 'details', 'text', 'category1'],
      dtype='object')


,timestamp,scrape_ts,domain,rooms,cost,address,duration,bid_ask,href,title,details,text,category1
0,2020-02-09 10:53:00,2020-10-19 17:21:49.163922,ronorp,1.0,1095.0,"Marktgasse 19, 8038 Kloten",unbefristet,Biete,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...","Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Agglo Vertragsart: unbefristet Zimmer: 1Kosten: 1'095.00Adresse: Marktgasse 19, 8038 Kloten Kontakt: i********@t********************h","Ich glaube, dass zusammen einfach schöner ist als alleine. Deshalb habe ich ein Start-up gegründet, das Gebäude für grosse Hausgemeinschaften baut.\nIn Kloten eröffnen wir nun am 1.Oktober unser erstes Gebäude mit 38 Suiten, komplett mit jeweils eigenem Bad (Dusche & WC). Die meisten Suiten habe...",Wohnung / Zimmer
1,2020-10-19 17:17:00,2020-10-19 17:21:51.717955,ronorp,1.0,600.0,"Idaplatz 2, 8003 Zürich",befristet,Biete,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-am-idaplatz-bis-ende-juni-2022.2074090?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,WG-Zimmer am Idaplatz bis Ende Juni 2022,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: befristet (auf Zeit) Zimmer: 1Kosten: 600.00Adresse: Idaplatz 2, 8003 Zürich Kontakt: w*****@g***********m","Per Februar 2021 wird in unserer 3er WG direkt am Idaplatz ein Zimmer frei. Die 4-Zimmerwohnung befindet sich im 2ten Stock mit zwei Balkone, einen in den ruhigen Innenhof, der andere direkt über dem Idaplatz. Zudem gibt es ein Wohnzimmer, eine Geschirrspülmaschine, zwei Kühl- & Gefrierschränke ...",Wohnung / Zimmer
2,2020-10-19 15:01:00,2020-10-19 17:21:54.263640,ronorp,1.0,850.0,"Üetlibergstrasse 139, 8045 Zürich",unbefristet,Biete,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-wiedikon-ab-november.2073817?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Zimmer in Wiedikon ab November,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 1Kosten: 850.00Besichtigungstermin: 22.10.2020, 18:00Adresse: Üetlibergstrasse 139, 8045 Zürich Kontakt:","Möbliertes Zimmer in ruhiger Lage, Wiedikon / Binz zu vermieten. Grüne, gemütliche Altbau Wohnung mit grossem Balkon & Wohnzimmer.",Wohnung / Zimmer
3,2020-10-19 14:50:00,2020-10-19 17:21:57.029696,ronorp,4.0,415.0,8037,unbefristet,Biete,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg50plus.2073814?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,WG50plus,Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 4Kosten: 415.00Adresse: 8037 Kontakt:,"WG50plus sucht 2 bis 3 Mitbewohner/innen In unserem grossen schönen 8-Zimmer-WG-Haus in Zürich-Wipkingen werden vier Zimmer frei. Wir wünschen uns zwei bis drei achtsame und humorvolle Mitbewohner/innen ab 50J., die mehr als eine günstige Bleibe suchen. Gemeinsam nutzen wir Küche, Ess- und Wohnz...",Wohnung / Zimmer
4,2020-10-19 14:38:00,2020-10-19 17:22:00.053402,ronorp,1.0,550.0,"Brunnmattstrasse 24, 3005 Bern",unbefristet,Biete,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zentrales-wg-zimmer-frei-mit-aussicht-buesis-ab-01-11-2020.2073812?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,zentrales WG-Zimmer frei mit Aussicht & Büsis ab...,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 1Kosten: 550.00Adresse: Brunnmattstrasse 24, 3005 Bern Kontakt: a*****@g***************m","Hier hast du a

In [89]:
# clean up df manually
# import dateparser
# df['scrape_ts']=df['scrape_ts'].apply(lambda x: dateparser.parse(x).replace(microsecond=0))
# df=df[['title', 'domain', 'rooms', 'cost', 'address', 'duration', 'href',
#         'timestamp', 'scrape_ts', 'processed','details','text']]
df=df[['timestamp', 'scrape_ts', 'domain', 'rooms', 'cost', 'address', 'duration', 
                   'bid_ask', 'href', 'title', 'details', 'text', 'category1' ]]
df.to_csv(fname,index=False)

In [74]:

textpatterns=[
    r'mehralswohnen',
    r'dialogweg',
    r'kraftwerk',
    r'hochfoif',
    r'mehr als 11',
    r'mehrals11',
    r'd6s4-SUED',
    r'Hunzkomune',
    r'Bunter Haufen',
    r'BunterHaufen',
    r'BuntGemischt',
    r'Bunt-Gemischt',
    r'Brückenwohnen',
#     r'MIR',
    r'HUNZREAL',
    r'kalkbreite',
    r'hunziker',
    r'Mehrgeneration',
    r'kraftwerk1',
    r'suite55',
    r'suite37',
    r'gross-wg',
    
]

addresspatterns=[
    r'dialogweg',
    r'hunziker'
]

titlepatterns = textpatterns


searchdict=[
    *[{'column':'text', 'pattern':pat,'flag':'IGNORECASE'} for pat in textpatterns],
    *[{'column':'address', 'pattern':pat,'flag':'IGNORECASE'} for pat in addresspatterns],
    *[{'column':'title', 'pattern':pat,'flag':'IGNORECASE'} for pat in titlepatterns]
#     {'column':'address', 'pattern': r'dialogweg','flag':'IGNORECASE'},
#     {'column':'address', 'pattern': r'hunziker','flag':'IGNORECASE'},
    
    
    
]
searchdict

[{'column': 'text', 'pattern': 'mehralswohnen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'dialogweg', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'kraftwerk', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'hochfoif', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'mehr als 11', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'mehrals11', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'd6s4-SUED', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Hunzkomune', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Bunter Haufen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'BunterHaufen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'BuntGemischt', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Bunt-Gemischt', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'Brückenwohnen', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'HUNZREAL', 'flag': 'IGNORECASE'},
 {'column': 'text', 'pattern': 'kalkbre

In [75]:
def make_clickable_both(val): 
    return f'<a href="{val}">link</a>' 

results=[]
for search in searchdict:
    col=search['column']
    pat=search['pattern']
    if search['flag'] == 'IGNORECASE':
        results.append(df[df[col].str.contains(pat,flags=re.IGNORECASE, regex=True)] )
    else:
        results.append(df[df[col].str.contains(pat, regex=True)] )
        
results2=pd.concat(results).drop_duplicates(subset=['title'])
showcols=['timestamp','scrape_ts','domain', 'rooms', 'cost', 'address','duration','title','href']
results2[showcols].style.format({'href': make_clickable_both,'cost':"{0:.0f}",'rooms':'{0:g}'})


,timestamp,scrape_ts,domain,rooms,cost,address,duration,title,href
35,2020-10-15 16:22:00,2020-10-19 17:28:14.263261,ronorp,5.5,590,"Dialogweg 3, 8001 Zürich",unbefristet,Room vacant in friendly shared household in a cooperative...,link
41,2020-10-14 21:11:00,2020-10-19 17:28:30.883825,ronorp,1,900,"Dialogweg 6, 8050 Zürich",unbefristet,Zimmer in Gross-WG auf dem Hunziker Areal per 1. Dezember...,link
112,2020-04-10 17:11:00,2020-10-19 17:33:46.879974,ronorp,1,952,"Kalkbreitestrasse 6, 8003 Zürich",unbefristet,Sechsköpfige Familien-WG mit zwei kleinen Kindern sucht...,link
182,2020-09-13 20:56:00,2020-10-19 17:39:08.081832,ronorp,2,nan,8003 Zürich,unbefristet,Familien-WG gesucht,link
248,2020-10-22 17:00:00,2020-10-22 21:49:18,ronorp,1,410,"Glatthofstrasse 6, 8152 Glattbrugg",unbefristet,Gross-WG in Glattbrugg sucht ab 1. Januar neue...,link


In [73]:
df

,title,domain,rooms,cost,address,duration,href,bid_ask,details,text,category1,timestamp,scrape_ts
0,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",ronorp,1.0,1095.0,"Marktgasse 19, 8038 Kloten",unbefristet,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,Biete,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Agglo Vertragsart: unbefristet Zimmer: 1Kosten: 1'095.00Adresse: Marktgasse 19, 8038 Kloten Kontakt: i********@t********************h","Ich glaube, dass zusammen einfach schöner ist als alleine. Deshalb habe ich ein Start-up gegründet, das Gebäude für grosse Hausgemeinschaften baut.\nIn Kloten eröffnen wir nun am 1.Oktober unser erstes Gebäude mit 38 Suiten, komplett mit jeweils eigenem Bad (Dusche & WC). Die meisten Suiten habe...",Wohnung / Zimmer,2020-02-09 10:53:00,2020-10-19 17:21:49.163922
1,WG-Zimmer am Idaplatz bis Ende Juni 2022,ronorp,1.0,600.0,"Idaplatz 2, 8003 Zürich",befristet,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-am-idaplatz-bis-ende-juni-2022.2074090?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,Biete,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: befristet (auf Zeit) Zimmer: 1Kosten: 600.00Adresse: Idaplatz 2, 8003 Zürich Kontakt: w*****@g***********m","Per Februar 2021 wird in unserer 3er WG direkt am Idaplatz ein Zimmer frei. Die 4-Zimmerwohnung befindet sich im 2ten Stock mit zwei Balkone, einen in den ruhigen Innenhof, der andere direkt über dem Idaplatz. Zudem gibt es ein Wohnzimmer, eine Geschirrspülmaschine, zwei Kühl- & Gefrierschränke ...",Wohnung / Zimmer,2020-10-19 17:17:00,2020-10-19 17:21:51.717955
2,Zimmer in Wiedikon ab November,ronorp,1.0,850.0,"Üetlibergstrasse 139, 8045 Zürich",unbefristet,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zimmer-in-wiedikon-ab-november.2073817?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Biete,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 1Kosten: 850.00Besichtigungstermin: 22.10.2020, 18:00Adresse: Üetlibergstrasse 139, 8045 Zürich Kontakt:","Möbliertes Zimmer in ruhiger Lage, Wiedikon / Binz zu vermieten. Grüne, gemütliche Altbau Wohnung mit grossem Balkon & Wohnzimmer.",Wohnung / Zimmer,2020-10-19 15:01:00,2020-10-19 17:21:54.263640
3,WG50plus,ronorp,4.0,415.0,8037,unbefristet,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg50plus.2073814?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Biete,Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 4Kosten: 415.00Adresse: 8037 Kontakt:,"WG50plus sucht 2 bis 3 Mitbewohner/innen In unserem grossen schönen 8-Zimmer-WG-Haus in Zürich-Wipkingen werden vier Zimmer frei. Wir wünschen uns zwei bis drei achtsame und humorvolle Mitbewohner/innen ab 50J., die mehr als eine günstige Bleibe suchen. Gemeinsam nutzen wir Küche, Ess- und Wohnz...",Wohnung / Zimmer,2020-10-19 14:50:00,2020-10-19 17:21:57.029696
4,zentrales WG-Zimmer frei mit Aussicht & Büsis ab...,ronorp,1.0,550.0,"Brunnmattstrasse 24, 3005 Bern",unbefristet,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/zentrales-wg-zimmer-frei-mit-aussicht-buesis-ab-01-11-2020.2073812?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Biete,"Biete / Suche / Tausche: Wohnung / Zimmer: Biete Mieten & Kaufen: Mieten Stadt / Agglo: Stadt Vertragsart: unbefristet Zimmer: 1Kosten: 550.00Adresse: Brunnmattstrasse 24, 3005 Bern Kontakt: a*****@g***************m","Hier hast du alles in der Nähe: Coop und Denner direkt im Geb